In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [3]:
import gradio as gr
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score
import sklearn.metrics as metrics

from joblib import dump
import os
import warnings
warnings.filterwarnings('ignore')

In [4]:
c_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gradio/Telco-Customer-Churn.csv')
c_data.drop(['customerID','PhoneService','SeniorCitizen','StreamingMovies','StreamingTV'], axis=1, inplace=True)
c_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   Partner           7043 non-null   object 
 2   Dependents        7043 non-null   object 
 3   tenure            7043 non-null   int64  
 4   MultipleLines     7043 non-null   object 
 5   InternetService   7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  Contract          7043 non-null   object 
 11  PaperlessBilling  7043 non-null   object 
 12  PaymentMethod     7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(1), object(14)
memory

In [5]:
c_data['TotalCharges'] = pd.to_numeric(c_data['TotalCharges'], errors='coerce')
c_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   Partner           7043 non-null   object 
 2   Dependents        7043 non-null   object 
 3   tenure            7043 non-null   int64  
 4   MultipleLines     7043 non-null   object 
 5   InternetService   7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  Contract          7043 non-null   object 
 11  PaperlessBilling  7043 non-null   object 
 12  PaymentMethod     7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7032 non-null   float64
 15  Churn             7043 non-null   object 
dtypes: float64(2), int64(1), object(13)
memory

In [6]:
# Removing missing values
c_data.dropna(inplace=True)
c_data.duplicated().sum()

26

In [7]:
#le = LabelEncoder()
#c_data['Churn'] = le.fit_transform(c_data['Churn'])
#c_data.head()

In [8]:
for col in c_data.columns:
    print(f"Column '{col}' categories: {c_data[col].unique()}")

Column 'gender' categories: ['Female' 'Male']
Column 'Partner' categories: ['Yes' 'No']
Column 'Dependents' categories: ['No' 'Yes']
Column 'tenure' categories: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
Column 'MultipleLines' categories: ['No phone service' 'No' 'Yes']
Column 'InternetService' categories: ['DSL' 'Fiber optic' 'No']
Column 'OnlineSecurity' categories: ['No' 'Yes' 'No internet service']
Column 'OnlineBackup' categories: ['Yes' 'No' 'No internet service']
Column 'DeviceProtection' categories: ['No' 'Yes' 'No internet service']
Column 'TechSupport' categories: ['No' 'Yes' 'No internet service']
Column 'Contract' categories: ['Month-to-month' 'One year' 'Two year']
Column 'PaperlessBilling' categories: ['Yes' 'No']
Column 'PaymentMethod' categories: ['Electronic check' 'Mailed check' 'Bank transfer 

In [9]:
y = c_data['Churn']                         # Target Variable
X = c_data.drop('Churn', axis =1)          # Independent Variable

In [10]:
numeric_transformer = Pipeline(steps = [('imputer',SimpleImputer(strategy = 'mean')),('scaler',StandardScaler())])
categorical_transformer = Pipeline(steps = [('imputer',SimpleImputer(strategy = 'most_frequent')),('one_hot_encoder',OneHotEncoder(handle_unknown='ignore', categories='auto', sparse=False))])

In [11]:
data_numeric =['tenure','MonthlyCharges','TotalCharges']
data_categorical =['gender','Partner','Dependents','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','Contract','PaperlessBilling','PaymentMethod']
preprocessor =ColumnTransformer(transformers =[('numeric',numeric_transformer,data_numeric),('categoric',categorical_transformer,data_categorical)])

In [12]:
# Identify numeric and non-numeric columns
#num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
#cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

#X_cat = X[cat_cols].copy()
#X_num = X[num_cols].copy()


In [13]:
# Creating imputer variables
#numerical_imputer = SimpleImputer(strategy = "mean")
#categorical_imputer = SimpleImputer(strategy = "most_frequent")



# Define the column transformer
#categorical_features = cat_cols
#categorical_transformer = Pipeline(steps=[('ohc', OneHotEncoder(handle_unknown='ignore', categories='auto', sparse=False))])
#preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, cat_cols)])


In [14]:
# Fitting the Imputer
#X_cat_imputed = categorical_imputer.fit_transform(X_cat)
#X_num_imputed = numerical_imputer.fit_transform(X_num)

In [15]:
#ohe = OneHotEncoder(handle_unknown='ignore')
#X_cat_encoded = ohe.fit(X_cat_imputed)
#X_cat_encoded = pd.DataFrame(ohe.transform(X_cat_imputed).toarray(),
#                                   columns=ohe.get_feature_names_out(cat_cols))



In [16]:
#scaler = StandardScaler()
#X_num_scaled = scaler.fit_transform(X_num_imputed)
#X_num_sc = pd.DataFrame(X_num_scaled, columns = num_cols)



In [17]:
#X_df = pd.concat([X_num_sc,X_cat_encoded], axis =1)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [19]:
gbc = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 5, n_estimators=50, random_state=30)
gbc = Pipeline(steps =[('processor',preprocessor),('estimator',gbc)])
model = gbc.fit(X_train,y_train)

# make predictions on the test data
y_pred = model.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred)
name = 'Gradient Boosting Classifier'
# print the classification report
print(f'{name} classification report:\n{report}\n')


Gradient Boosting Classifier classification report:
              precision    recall  f1-score   support

          No       0.83      0.89      0.86      1031
         Yes       0.64      0.51      0.57       376

    accuracy                           0.79      1407
   macro avg       0.74      0.70      0.72      1407
weighted avg       0.78      0.79      0.78      1407




In [20]:
  # create a dictionary of a model to fit
  #models = {'Gradient Boosting Classifier': GradientBoostingClassifier(learning_rate = 0.1, max_depth = 5, n_estimators=50, random_state=30),}
  # iterate over the models and fit each one to the resampled training data
  #for name, model in models.items():
  #  model.fit(X_train, y_train)

  # evaluate each model using cross-validation based on ROC-AUC
  #roc_auc_scores = {}
  #for name, model in models.items():
  #  scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
  #  roc_auc_scores[name] = scores.mean()

  # print the ROC-AUC scores for each model
  #for name, score in roc_auc_scores.items():
  #  print(f'{name}: {score}')

  # choose the model with the highest ROC-AUC score
  #best_model_name = max(roc_auc_scores, key=roc_auc_scores.get)
  #best_model = models[best_model_name]
  #print(f'Best model: {best_model_name}')


  # iterate over the models and make predictions on the test data for each one
  #for name, model in models.items():
    # fit the model to the resampled training data
  #  model.fit(X_train, y_train)
    # make predictions on the test data
  #  y_pred = model.predict(X_test)
    # generate the classification report
  #  report = classification_report(y_test, y_pred)
    # print the classification report
 #   print(f'{name} classification report:\n{report}\n')




In [21]:
# set the destination path to the "export" directory
destination = "."

#best_model = gbc
models = {"best_model": gbc}

# loop through the models and save them using joblib.dump()
for name, model in models.items():
    dump(model, os.path.join(destination, f"{name}.joblib"))


In [22]:
!pip freeze > requirements.txt

In [23]:
#for name, model in models.items():
#    dump(model, os.path.join(destination, f"{name}.joblib"))
#    if os.path.exists(os.path.join(destination, f"{name}.joblib")):
#        print(f"{name} saved successfully!")
#    else:
#        print(f"{name} failed to save.")